# Main Function

Need to do the following in terminal to run:
mkdir tmp
mkdir tmp/ppo
mkdir plots


In [1]:
import gym
import torch
import numpy as np
from ipynb.fs.full.CartPolePPO import Agent
from utils import plot_learning_curve

In [5]:
if __name__ == '__main__':
    env = gym.make('CartPole-v1')
    N = 20
    batch_size = 5
    n_epochs = 4
    alpha = 0.0003

    agent = Agent(num_actions = env.action_space.n, batch_size = 5, alpha = 0.0003, num_epochs = 4, input_dims = env.observation_space.shape)
    n_games = 200

    figure_file = 'plots/CartPole.png'

    best_score = env.reward_range[0]
    score_history = []

    learn_iters = 0
    avg_score = 0
    n_steps = 0

    agent.load_models()

    for i in range(n_games): # 103 min 8.8 sec for 300 iterations
        observation = env.reset()
        # observation = prepro(observation)
        done = False
        score = 0
        while not done:
            action, prob, val = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            n_steps += 1
            score += reward
            agent.remember(observation, action, prob, val, reward, done)
            if n_steps % N == 0: # if true, it's time to perform learning function
                agent.learn()
                learn_iters += 1
            observation = observation_
        score_history.append(score)
        avg_score = np.mean(score_history[-100:])

        if avg_score > best_score: # if best score found
            best_score = avg_score
            agent.save_models()

        print('episode', i, 'score %.1f' % score, 'avg score %.1f' % avg_score,
                'time_steps', n_steps, 'learning_steps', learn_iters)
    x = [i+1 for i in range(len(score_history))]
    plot_learning_curve(x, score_history, figure_file, "Training Episodes", "Average Scores")


-0.616603434085846
-0.7798265218734741
-0.6166165471076965
-0.6127517223358154
-0.610337495803833
-0.6108036041259766
-0.7815525531768799
-0.7819428443908691
-0.6105459928512573
-0.7819991111755371
-0.7832805514335632
-0.6102901697158813
-0.6102262139320374
-0.7822152972221375
... saving models ...
episode 0 score 14.0 avg score 14.0 time_steps 14 learning_steps 0
-0.6166999340057373
-0.7772564888000488
-0.7756340503692627
-0.7733612060546875
-0.7863333225250244
-0.5946796536445618
-0.565714955329895
-0.5572540760040283
-0.8569232821464539
-0.5584623217582703
-0.5515159368515015
-0.5809540748596191
-0.7721621990203857
-0.57607501745224
-0.7780447602272034
-0.5718904137611389
-0.6100406050682068
-0.735765278339386
-0.6057841777801514
-0.739749014377594
-0.6027463674545288
-0.6458286046981812
-0.6831061244010925
-0.644019365310669
-0.6836256384849548
-0.643721342086792
-0.6901944279670715
-0.5963711142539978
-0.6953819990158081
-0.806967556476593
-0.9134504199028015
-0.44728749990463257


KeyboardInterrupt: 

# Save Trained Model

In [4]:
# agent.save_models()

... saving models ...


# Load Trained Model and Run Iteration

In [ ]:
# Load model
agent.load_models()

In [4]:
env = gym.make('CartPole-v1')
env.reset()
score = 0
done = False
n_steps = 0
while not done:
    env.render()
    action, prob, val = agent.choose_action(observation)
    observation_, reward, done, info = env.step(action)
    n_steps += 1
    score += reward
    agent.remember(observation, action, prob, val, reward, done)
    if n_steps % N == 0: # if true, it's time to perform learning function
        agent.learn()
        learn_iters += 1
    observation = observation_
print(score)
env.close()


79.0
